# MySQL 8.0.40

## 0) 前提

* エンジン: **MySQL 8**
* 並び順: 任意（`ORDER BY` なし）
* `NOT IN` は NULL 罠のため回避
* 判定は **ID 基準**（`referee_id` が 2 か否か）、表示は仕様どおりの列名と順序（`name`）

## 1) 問題

* `{{PROBLEM_STATEMENT}}`
  「紹介者が id=2 **以外**の顧客、または **未紹介（NULL）** の顧客の **name** を返せ。」
* 入力テーブル例: `{{TABLES_OR_SCHEMAS}}`
  `Customer(id INT PK, name VARCHAR, referee_id INT NULL)`
* 出力仕様: `{{OUTPUT_COLUMNS_AND_RULES}}`
  列は `name` のみ。順序は任意。重複は想定なし（`id` が PK で 1 行=1 顧客）。

## 2) 最適解（単一クエリ）

> この問題はウィンドウ不要。**NULL を明示的に許容**し、id=2 を除外するだけで良い。

```sql
-- MySQL 8
SELECT name
FROM Customer
WHERE referee_id IS NULL
   OR referee_id <> 2;

-- Runtime 579 ms
-- Beats 13.38%

```

* `IS NULL` で未紹介を拾い、`<> 2` で **紹介者が 2 ではない**行を拾う。
* `NOT IN (2)` は `referee_id` に NULL があると空振りする可能性があるため不採用。

## 3) 代替解

> `EXISTS` / `LEFT JOIN ... IS NULL` を使った **NULL 安全**な書き方。

```sql
-- A) NOT EXISTS 版（「紹介者=2 という事実が存在しない」）
SELECT c.name
FROM Customer AS c
WHERE NOT EXISTS (
  SELECT 1
  FROM Customer AS r
  WHERE r.id = 2
    AND c.referee_id = r.id
);

-- Runtime 450 ms
-- Beats 63.44%

-- B) LEFT JOIN 版（マッチできなければ条件に合致）
SELECT c.name
FROM Customer AS c
LEFT JOIN Customer AS r
  ON r.id = 2 AND c.referee_id = r.id
WHERE r.id IS NULL;

-- Runtime 462 ms
-- Beats 52.60%

```

* いずれも `NOT IN` を使わず **NULL 影響を受けない**。

## 4) 要点解説

* **方針**: 条件は「`referee_id` が **NULL** または **2 以外**」。追加のユニーク化や順位付けは不要。
* **NULL / 重複**:

  * `referee_id IS NULL` を必ず明示。`referee_id <> 2` だけだと `NULL` 行は落ちる（3値論理）。
  * 行重複は入力が 1 顧客 1 行で起きないため `DISTINCT` 不要。
* **安定性**: 順序自由のため `ORDER BY` は付けない方が高速。

## 5) 計算量（概算）

* **単一テーブル・単一述語**なので全表走査で **O(N)**。
  `referee_id` にインデックスがあれば **O(N)** から **O(M)** へ近似的に短縮（M は該当行数）。ただし `IS NULL` と `<> 2` の複合条件で最適化はデータ分布次第。

## 6) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[Customer 全行] --> B[条件: referee_id IS NULL]
  A --> C[条件: referee_id <> 2]
  B --> D[OR で合流]
  C --> D[OR で合流]
  D --> E[投影 name のみ]
  E --> F[出力]
```

---

### サンプルに対する結果（確認）

入力例では `Will(Janeの紹介なし)`, `Jane(NULL)`, `Bill(NULL)`, `Zack(referee_id=1)` が該当し、出力は問題例と一致します。


以下は **マークダウン**での改善クエリ例です（上から順におすすめ）。すべて **MySQL 8**想定です。

---

### ✅ 改善案A：ORを除去（NULLセーフ等価）

`IS NULL OR <>` を 1 述語にまとめ、最もシンプル＆高速化しやすい形にします。

```sql
-- 改善案A（推奨）
SELECT name
FROM Customer
WHERE NOT (referee_id <=> 2);

-- Runtime 415 ms
-- Beats 92.07%

```

---

### ✅ 改善案B：UNION ALL で枝分け（index_merge/レンジ最適化を誘発）

`IS NULL` と `<> 2` は重複しないため分割可能。プラン次第で速くなることがあります。

```sql
-- 改善案B
SELECT name
FROM Customer
WHERE referee_id IS NULL
UNION ALL
SELECT name
FROM Customer
WHERE referee_id <> 2;

-- Runtime 498 ms
-- Beats 28.46%

```

---

### ✅ 改善案C：不等号を範囲に分解（<> を < と > に）

レンジ条件に分けることでレンジスキャンや index_merge(union) が選ばれることがあります。

```sql
-- 改善案C
SELECT name
FROM Customer
WHERE referee_id IS NULL
   OR referee_id < 2
   OR referee_id > 2;

-- Runtime 482 ms
-- Beats 37.21%
```

---

## 併用すると効くインデックス（I/O削減：カバリング）

返す列が `name` だけなので、以下の **カバリングインデックス**でテーブルへ戻らずに完結でき、I/O が減ります。

```sql
CREATE INDEX idx_customer_referee_name ON Customer(referee_id, name);
```

> 上のインデックスは **A/B/C どれにも有効**です。まずは A（`NOT (referee_id <=> 2)`）＋このインデックスの組み合わせを試すのが定石です。

---

## 実行計画の確認（統計更新 → 実測）

```sql
ANALYZE TABLE Customer;

EXPLAIN ANALYZE
SELECT name
FROM Customer
WHERE NOT (referee_id <=> 2);
```

必要に応じて B/C と比較して、最短の実行時間・最小の行読み取りを採用してください。
